# <a> [Lighthouse] Desafio Cientista de Dados </a>

**Motivação**

Este desafio proposto pela INDICIUM representa uma excelente oportunidade para colocar em prática meus conhecimentos em resolução de problemas de negócios, análise de dados e desenvolvimento de modelos preditivos.

**Contexto do Desafio**

“Você foi alocado(a) em um time da Indicium que está trabalhando atualmente junto a um cliente que o core business é compra e venda de veículos usados. Essa empresa está com dificuldades na área de revenda dos automóveis usados em seu catálogo. Para resolver esse problema, a empresa comprou uma base de dados de um marketplace de compra e venda para entender melhor o mercado nacional, de forma a conseguir precificar o seu catálogo de forma mais competitiva e assim recuperar o mau desempenho neste setor. Seu objetivo é analisar os dados para responder às perguntas de negócios feitas pelo cliente e criar um modelo preditivo que precifique os carros do cliente de forma que eles fiquem o mais próximos dos valores de mercado.”

**Objetivo**

Analisar os dados disponíveis para responder de forma assertiva às perguntas de negócio formuladas pelo cliente e, com base nessa análise, desenvolver um modelo preditivo capaz de estimar os preços dos veículos de maneira competitiva, aproximando-os ao máximo dos valores praticados no mercado.

# Notebook: modelagem preditiva (modelagem.ipynb)

**- 1.Carregar os dados:** 
    
    - Ler os arquivos cars_training.csv e cars_test.csv.

***- 2.Pré-processamento dos dados:***
    
    - Criar uma cópia independente do DataFrame.
    - Remover a coluna 'id' e 'veiculo_alienado' do conjunto de features.
    - Converte valores categóricos (como texto) em valores numéricos.

***- 3.Treinamento do modelo:***
    
    - Separar as features (X) e a variável alvo (y) do dataset de treinamento.
    - Criar um modelo de regressão (usaremos o RandomForestRegressor).
    - Considera a média dos erros absolutos de um grupo de previsões e observações como uma medida da magnitude dos erros para todo o grupo.
    
***- 4.Previsão no dataset de teste:***
    
    - Fazer previsões na coluna 'preco' do dataset de teste.
    
***- 5.Salvar o resulta (opcional):***
    
    - Salvar este DataFrame em um arquivo CSV chamado 'precos_previstos.csv'.

### Índice
- [1. Importando Bibliotecas e Configurando Pandas](#1.)

- [2. Carregar os dados](#2.)

- [3. Pré-processamento (treino)](#3.)

- [4. Treinamento do modelo](#4.)

- [5. Resultado](#5.)


## <a id='1.'></a>1. Importando Bibliotecas e Configurando Pandas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
# configurando pandas para não mostrar notação científica para números
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# configurando pandas para ignorar avisos
import warnings
warnings.filterwarnings('ignore')

## <a id='2.'></a>2. Carregar os dados

In [4]:
# Carregar os dados
df_train = pd.read_csv('data/cars_training.csv', encoding='utf-16', sep='\t')
df_test = pd.read_csv('data/cars_test.csv', encoding='utf-16', sep='\t')

## <a id='3.'></a>3. Pré-processamento (treino)

In [5]:
#Pré-processamento (treino)
df = df_train.copy()

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna('Não informado')

for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(df[col].median())

df = df.drop(columns=['id', 'veiculo_alienado'])

label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


## <a id='4.'></a>4. Treinamento do modelo

In [6]:
# Treinamento do modelo
X = df.drop(columns=['preco'])
y = df['preco']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'Erro médio absoluto (MAE): R$ {mae:,.2f}')

Erro médio absoluto (MAE): R$ 26,778.80


## <a id='5.'></a>5. Preparação do teste + Previsão

In [7]:
# Preparação do teste + Previsão
df_test_clean = df_test.copy()

for col in df_test_clean.select_dtypes(include='object').columns:
    df_test_clean[col] = df_test_clean[col].fillna('Não informado')

for col in df_test_clean.select_dtypes(include=['float64', 'int64']).columns:
    df_test_clean[col] = df_test_clean[col].fillna(df_test_clean[col].median())

df_test_clean = df_test_clean.drop(columns=['id', 'veiculo_alienado'])

for col, le in label_encoders.items():
    if col in df_test_clean.columns:
        df_test_clean[col] = df_test_clean[col].apply(lambda x: x if x in le.classes_ else 'Não informado')
        if 'Não informado' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Não informado')
        df_test_clean[col] = le.transform(df_test_clean[col])


## <a id='6.'></a>6. Resultado

In [8]:
# Resultado final
precos_previstos = model.predict(df_test_clean)
df_test['preco_previsto'] = precos_previstos
df_test[['marca', 'modelo', 'versao', 'ano_de_fabricacao', 'hodometro', 'preco_previsto']].head()


,marca,modelo,versao,ano_de_fabricacao,hodometro,preco_previsto
0,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,20258.00,75125.42
1,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,53045.00,86916.28
2,AUDI,Q5,2.0 TFSI GASOLINA BLACK S TRONIC,2018,32486.00,244988.59
3,CHEVROLET,CRUZE,1.4 TURBO LT 16V FLEX 4P AUTOMÁTICO,2019,24937.00,103481.53
4,FORD,ECOSPORT,1.5 TI-VCT FLEX SE AUTOMÁTICO,2019,62074.00,105435.23


In [9]:
# (Opcional) Salvar os resultados
df_test.to_csv('data/precos_previstos.csv', index=False, encoding='utf-8')